In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')
# contracts = populate_contracts(tickers)

In [62]:
from stockMarket.sec_edgar.downloader import Downloader

tickers = [ticker.replace(".", "-") for ticker in tickers]

dl = Downloader(tickers=tickers)
dl.download_10k_reports(fast_download=True)
dl.download_financial_statements_xlsx(from_year=2014)

 50%|████▉     | 249/503 [00:00<00:00, 2488.53it/s]/home/jag/projects/stockMarket/stockMarket/sec_edgar/downloader.py:51: UserWarning: No 10-K reports found for KVUE with cik 0001944048
  warnings.warn(message, UserWarning)
100%|██████████| 503/503 [00:00<00:00, 2730.21it/s]


In [177]:
import glob

from stockMarket.sec_edgar.financialReportReader import FinancialReportReader
from tqdm import tqdm
from zipfile import BadZipFile

key_combinations = []

for ticker in tqdm(sorted(tickers)):
# for ticker in tqdm(sorted(tickers)[492:]):
    files = glob.glob(f"../data/sec-edgar-filings/{ticker}/10-K/*/Financial_Report.xlsx")

    for file in files:
        if 14 < int(file.split("-")[-2]) <= 24:
            try:
                reader = FinancialReportReader(file)
                reader.read()
                key_combinations += reader.key_combinations
            except BadZipFile as e:
                print(f"BadZipFile: {ticker}")
                raise e

  0%|          | 0/502 [00:00<?, ?it/s]

100%|██████████| 502/502 [00:05<00:00, 83.85it/s]


In [78]:
from zipfile import BadZipFile

from stockMarket.sec_edgar.financialReportCollecter import FinancialReportCollecter

tickers = [ticker.replace(".", "-") for ticker in tickers if ticker != "KVUE"]

new_tickers = []
new_tickers.clear()
for ticker in sorted(tickers):
    if ticker in ["ZION", "WFC", "USB", "TFC", "SYF", "RF", "NTRS", "L", "KEY", "HBAN", "FITB", "DFS", "COF", "CMA", "MTB", "NVR", "AXP", "BAC", "BK", "C", "CARR", "CDW", "CFG"]:
        continue
    
    if ticker in ["ABNB", "HPQ", "INVH", "O", "GEHC", "LIN", "ALLE", "HLT", "KEYS"]: # ABNB 2021 no data available and only 24, 23, 22 are available
        continue
    
    new_tickers.append(ticker)
    
collecter = FinancialReportCollecter(tickers=new_tickers)
collecter.collect(update=True)
collecter.error_messages

 22%|██▏       | 104/470 [11:37<30:38,  5.02s/it] /home/jag/Trading/stockMarket/stockMarket/sec_edgar/financialReportCollecter.py:95: UserWarning: CRM does not have all reports for the years 14 to 24
  warnings.warn(
 26%|██▌       | 123/470 [13:13<25:40,  4.44s/it]/home/jag/Trading/stockMarket/stockMarket/sec_edgar/financialReportCollecter.py:95: UserWarning: DHI does not have all reports for the years 14 to 24
  warnings.warn(
 42%|████▏     | 196/470 [22:29<14:40,  3.21s/it]  /home/jag/Trading/stockMarket/stockMarket/sec_edgar/financialReportCollecter.py:95: UserWarning: GPN does not have all reports for the years 14 to 24
  warnings.warn(
 48%|████▊     | 226/470 [25:01<19:32,  4.80s/it]/home/jag/Trading/stockMarket/stockMarket/sec_edgar/financialReportCollecter.py:95: UserWarning: IP does not have all reports for the years 14 to 24
  warnings.warn(
 54%|█████▍    | 253/470 [27:54<22:06,  6.11s/it]/home/jag/Trading/stockMarket/stockMarket/sec_edgar/financialReportCollecter.py:95: U

[]

In [73]:
import numpy as np

from stockMarket.sec_edgar.financialReportChecker import FinancialReportChecker
from tqdm import tqdm

correct_tickers = []

#correct until -18-
# checked_tickers = ["AAL", "ACN", "AEE", "AJG", "AKAM"]

# #correct until -19-
# checked_tickers += ["ADP", "AMAT"]

# checked_tickers += ["ACGL", "AFL", "AIG", "AIZ", "ALL"]

for ticker in tqdm(sorted(new_tickers)):
    try:
        checker = FinancialReportChecker(ticker)
        checked = checker.check_data()
    except Exception as e:
        print(f"Exception: {ticker}")
        raise e
    
    
    if not checked and ticker not in checked_tickers:
        print(ticker)
        print(checker.ref_revenue)
        print(checker.revenue[:len(checker.ref_revenue)])
        print(np.array([str(date) for date in checker.ref_dates]))
        print(checker.dates[:len(checker.ref_revenue)])
        break
    else:
        correct_tickers.append(ticker)
    

  5%|▌         | 24/465 [00:06<02:00,  3.66it/s]

AMD
[22680. 23601. 16434.  9763.  6731.  6475.  5253.]
[22680. 23601. 16434.  9763.  6731.  6475.  5329.]
['2023-12-30' '2022-12-31' '2021-12-25' '2020-12-26' '2019-12-28'
 '2018-12-29' '2017-12-30']
['2023-12-30' '2022-12-31' '2021-12-25' '2020-12-26' '2019-12-28'
 '2018-12-29' '2017-12-30']


In [244]:
for ticker in correct_tickers:
    print(ticker)

A
AAL
AAPL
ABBV
ABT
